In [1]:
import pandas as pd
import numpy as np
import cv2
import time
import os
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_colwidth', None)

In [2]:
os.path.exists('eu-seg/top_00000.txt')

True

In [3]:
def check_part(loc, no): # loc: eu, ha(jp), th | part: top, bottom, dress | no: file number
    partList = ['top', 'bottom', 'dress']
    partCheck = [0, 0, 0]
    # loc = loc + '-seg/'
    loc = loc + '-seg/test/'
    for i in range(len(partList)):
        path = loc + '{}_{:05d}.txt'.format(partList[i], no)
        if(os.path.exists(path)):
            partCheck[i] = 1
    return partCheck

In [4]:
def pixel_gathering(loc, part, no):
    list_pos = []
    # loc = loc + '-seg/'
    loc = loc + '-seg/test/'
    path = loc + '{}_{:05d}.txt'.format(part, no)
    with open(path, 'r') as filestream:
        count = 0
        for line in filestream:
            count = count + 1
            pos_i, pos_j = line.split(', ')
            pos_j = pos_j.replace('\n', '')
            list_pos.append((int(pos_i), int(pos_j)))
            # print('pos_i = {} | pos_j = {}'.format(pos_i, pos_j))
#         print('line count = {}'.format(count))
    return list_pos

In [5]:
def count_px(img, posList, color_range):
    
    rgb_count = np.array([[[0 for i in range(0, 256, color_range)] for i in range(0, 256, color_range)] for i in range(0, 256, color_range)])
    
    for [i, j] in posList:
        r_val = img[i, j, 0]
        g_val = img[i, j, 1]
        b_val = img[i, j, 2]

        r_idx = int(r_val/color_range)
        g_idx = int(g_val/color_range)
        b_idx = int(b_val/color_range)

        # print('r: {}, g: {}, b: {}'.format(r_idx, g_idx, b_idx))
        rgb_count[r_idx, g_idx, b_idx] = rgb_count[r_idx, g_idx, b_idx]+1

    count_list = []
    for i in range(color_range):
        for j in range(color_range):
            for k in range(color_range):
                count_list.append(rgb_count[i, j, k])
    
    return count_list

In [6]:
def colorfulness(img, posList):
    rg_ch = 0 # red-green channel
    by_ch = 0 # blue-yellow channel
    rg_list = []
    by_list = []
    for [i, j] in posList:
        rg_ch = rg_ch + np.abs(int(img[i, j, 0]) - int(img[i, j, 1]))
        by_ch = by_ch + np.abs((0.5) * (int(img[i, j, 0]) + int(img[i, j, 1])) - int(img[i, j, 2]))
        rg_list.append(np.abs(int(img[i, j, 0]) - int(img[i, j, 1])))
        by_list.append(np.abs((0.5) * (int(img[i, j, 0]) + int(img[i, j, 1])) - int(img[i, j, 2])))
#         rg_ch = rg_ch + (img[i, j, 0] - img[i, j, 1])
#         by_ch = by_ch + ((0.5) * (img[i, j, 0] + img[i, j, 1]) - img[i, j, 2])
#         rg_list.append((img[i, j, 0] - img[i, j, 1]))
#         by_list.append(((0.5) * (img[i, j, 0] + img[i, j, 1]) - img[i, j, 2]))
        
    u_rg = rg_ch / len(posList)
    u_by = by_ch / len(posList)
    u_rgby = np.sqrt(u_rg**2 + u_by**2)
    std_rg = np.std(rg_list)
    std_by = np.std(by_list)
    std_rgby = np.sqrt(std_rg**2 + std_by**2)
    print('u_rg = {:.3f} | u_by = {:.3f} | u_rgby = {:.3f} | std_rg = {:.3f} | std_by = {:.3f} | std_rgby = {:.3f}'.
          format(u_rg, u_by, u_rgby, std_rg, std_by, std_rgby))
    return rg_list, by_list, std_rgby + 0.3*u_rgby

In [7]:
def transform_tab(tab, tab_init, top_n, filename, label):
    top = tab.sort_values('proportion', ascending=False).iloc[0:7].drop('count', axis=1)
    temp = tab_init.copy()
    temp['filename'] = [filename]
    for i in range(1, top_n+1):
        # print(i)
        temp['r_range_{}'.format(i)] = [top.iloc[i-1, 0]]
        temp['g_range_{}'.format(i)] = [top.iloc[i-1, 1]]
        temp['b_range_{}'.format(i)] = [top.iloc[i-1, 2]]
        temp['proportion_{}'.format(i)] = [top.iloc[i-1, 3]]
    temp['label'] = label
    return temp

In [8]:
top7_tab_init = pd.read_csv('Report/top7_count_tap.csv')
top7_tab_top = top7_tab_init.copy()
top7_tab_bottom = top7_tab_init.copy()
top7_tab_dress = top7_tab_init.copy()
count_tab_init = pd.read_csv('Report/count_tab_init.csv')
color_range = 16

In [9]:
# file_loc = location + file_name
st = time.perf_counter()
class_names = ['short_sleeved_shirt', 'long_sleeved_shirt', 'short_sleeved_outwear', 'long_sleeved_outwear', 'vest', 'sling',
               'shorts', 'trousers', 'skirt', 'short_sleeved_dress', 'long_sleeved_dress', 'vest_dress', 'sling_dress']
column_names = ['filename']+class_names
class_tab = pd.DataFrame(columns=column_names)
colorful_tab = pd.DataFrame(columns=['filename', 'colorfulness'])
top7_tab = top7_tab_init.copy()

for loc in ['eu', 'ha', 'th']:

# for loc in ['eu']:
    
    if(loc=='eu'):
        # file_loc_init = 'european-street-fashion-output/'
        file_loc_init = 'european-street-fashion-output/test/'
        label = 0
    elif(loc=='ha'):
        # file_loc_init = 'harajuku-kawaii-street-fashion-output/'
        file_loc_init = 'harajuku-kawaii-street-fashion-output/test/'
        label = 1
    elif(loc=='th'):
        # file_loc_init = 'thai-street-fashion-output/'
        file_loc_init = 'thai-street-fashion-output/test/'
        label = 0
        
    for i in range(1000):
        file_loc = file_loc_init+'test{:05d}.png'.format(i)
        print(file_loc)
        test = cv2.imread(file_loc)
        try:
            test = cv2.cvtColor(test, cv2.COLOR_BGR2RGB)
        except:
            continue
        count_tab = count_tab_init.copy()

        part = check_part(loc, i)
        # print(part)

        pos_dress = []
        pos_top = []
        pos_bottom = []
        if(np.sum(part)!=0):
            if(part[0]==1):
                pos_top = pixel_gathering(loc, 'top', i)
            if(part[1]==1):
                pos_bottom = pixel_gathering(loc, 'bottom', i)
            if(part[2]==1):
                pos_dress = pixel_gathering(loc, 'dress', i)
            
            f = open('{}-seg/test/class_{:05d}.txt'.format(loc, i), 'r')
            classProb = f.readlines()[0].split(' ')[:-1]
            class_names = ['short_sleeved_shirt', 'long_sleeved_shirt', 'short_sleeved_outwear', 'long_sleeved_outwear', 'vest', 'sling',
                           'shorts', 'trousers', 'skirt', 'short_sleeved_dress', 'long_sleeved_dress', 'vest_dress', 'sling_dress']
            
            data = ['{}'.format(file_loc)]+classProb
            tmp_class = pd.DataFrame(data=np.array(data, ndmin=2), columns=column_names)
            class_tab = pd.concat([class_tab, tmp_class], axis=0)
            
            temp = list(set.union(set(pos_dress), set(pos_top), set(pos_bottom)))
            temp.sort()
            pos_dress = temp.copy()

            rg_list, by_list, temp_colorfulness = colorfulness(test, pos_dress)
            data = ['{}'.format(file_loc), temp_colorfulness]
            tmp_tab = pd.DataFrame(data=np.array(data, ndmin=2), columns=['filename', 'colorfulness'])
            colorful_tab = pd.concat([colorful_tab, tmp_tab], axis=0)
            
            count_list = count_px(test, pos_dress, color_range)
            count_tab['count'] = count_list
            count_tab['proportion'] = count_tab['count']/sum(count_tab['count'])
            temp_top7_tab = transform_tab(count_tab, top7_tab_init, 7, file_loc, label)
            top7_tab = pd.concat([top7_tab, temp_top7_tab], axis=0)
    
    

    
ed = time.perf_counter() 

european-street-fashion-output/test/test00000.png
u_rg = 11.582 | u_by = 17.002 | u_rgby = 20.572 | std_rg = 16.271 | std_by = 24.449 | std_rgby = 29.368
european-street-fashion-output/test/test00001.png
u_rg = 3.519 | u_by = 5.443 | u_rgby = 6.481 | std_rg = 7.170 | std_by = 9.281 | std_rgby = 11.728
european-street-fashion-output/test/test00002.png
european-street-fashion-output/test/test00003.png
european-street-fashion-output/test/test00004.png
european-street-fashion-output/test/test00005.png
u_rg = 2.136 | u_by = 5.580 | u_rgby = 5.975 | std_rg = 4.823 | std_by = 5.382 | std_rgby = 7.227
european-street-fashion-output/test/test00006.png
u_rg = 39.764 | u_by = 37.005 | u_rgby = 54.319 | std_rg = 34.866 | std_by = 32.844 | std_rgby = 47.899
european-street-fashion-output/test/test00007.png
european-street-fashion-output/test/test00008.png
u_rg = 15.678 | u_by = 22.896 | u_rgby = 27.749 | std_rg = 9.492 | std_by = 14.410 | std_rgby = 17.255
european-street-fashion-output/test/test00

u_rg = 35.446 | u_by = 27.039 | u_rgby = 44.582 | std_rg = 23.442 | std_by = 19.294 | std_rgby = 30.361
european-street-fashion-output/test/test00080.png
european-street-fashion-output/test/test00081.png
european-street-fashion-output/test/test00082.png
european-street-fashion-output/test/test00083.png
u_rg = 32.932 | u_by = 33.617 | u_rgby = 47.060 | std_rg = 38.773 | std_by = 30.120 | std_rgby = 49.098
european-street-fashion-output/test/test00084.png
u_rg = 14.872 | u_by = 23.619 | u_rgby = 27.911 | std_rg = 10.014 | std_by = 12.112 | std_rgby = 15.716
european-street-fashion-output/test/test00085.png
u_rg = 44.076 | u_by = 45.986 | u_rgby = 63.698 | std_rg = 25.940 | std_by = 23.791 | std_rgby = 35.198
european-street-fashion-output/test/test00086.png
european-street-fashion-output/test/test00087.png
u_rg = 21.675 | u_by = 29.258 | u_rgby = 36.412 | std_rg = 25.884 | std_by = 32.666 | std_rgby = 41.678
european-street-fashion-output/test/test00088.png
u_rg = 10.398 | u_by = 21.066 

european-street-fashion-output/test/test00162.png
european-street-fashion-output/test/test00163.png
u_rg = 13.453 | u_by = 19.914 | u_rgby = 24.032 | std_rg = 11.977 | std_by = 18.926 | std_rgby = 22.398
european-street-fashion-output/test/test00164.png
u_rg = 6.568 | u_by = 5.326 | u_rgby = 8.456 | std_rg = 10.491 | std_by = 8.462 | std_rgby = 13.479
european-street-fashion-output/test/test00165.png
european-street-fashion-output/test/test00166.png
european-street-fashion-output/test/test00167.png
european-street-fashion-output/test/test00168.png
u_rg = 24.335 | u_by = 69.006 | u_rgby = 73.171 | std_rg = 20.787 | std_by = 22.391 | std_rgby = 30.552
european-street-fashion-output/test/test00169.png
u_rg = 9.906 | u_by = 18.707 | u_rgby = 21.168 | std_rg = 7.398 | std_by = 12.172 | std_rgby = 14.244
european-street-fashion-output/test/test00170.png
u_rg = 7.872 | u_by = 12.450 | u_rgby = 14.730 | std_rg = 7.526 | std_by = 8.429 | std_rgby = 11.300
european-street-fashion-output/test/tes

u_rg = 37.369 | u_by = 10.601 | u_rgby = 38.844 | std_rg = 13.016 | std_by = 8.342 | std_rgby = 15.460
european-street-fashion-output/test/test00244.png
u_rg = 23.767 | u_by = 8.552 | u_rgby = 25.259 | std_rg = 10.969 | std_by = 7.345 | std_rgby = 13.201
european-street-fashion-output/test/test00245.png
u_rg = 19.238 | u_by = 31.521 | u_rgby = 36.928 | std_rg = 15.826 | std_by = 21.001 | std_rgby = 26.297
european-street-fashion-output/test/test00246.png
european-street-fashion-output/test/test00247.png
u_rg = 8.590 | u_by = 3.371 | u_rgby = 9.228 | std_rg = 2.578 | std_by = 3.169 | std_rgby = 4.086
european-street-fashion-output/test/test00248.png
u_rg = 30.639 | u_by = 20.411 | u_rgby = 36.815 | std_rg = 22.177 | std_by = 12.840 | std_rgby = 25.625
european-street-fashion-output/test/test00249.png
european-street-fashion-output/test/test00250.png
u_rg = 17.688 | u_by = 23.347 | u_rgby = 29.291 | std_rg = 16.033 | std_by = 12.533 | std_rgby = 20.350
european-street-fashion-output/test

u_rg = 53.407 | u_by = 33.483 | u_rgby = 63.035 | std_rg = 28.843 | std_by = 15.029 | std_rgby = 32.524
european-street-fashion-output/test/test00335.png
european-street-fashion-output/test/test00336.png
european-street-fashion-output/test/test00337.png
european-street-fashion-output/test/test00338.png
european-street-fashion-output/test/test00339.png
european-street-fashion-output/test/test00340.png
european-street-fashion-output/test/test00341.png
european-street-fashion-output/test/test00342.png
u_rg = 29.491 | u_by = 185.920 | u_rgby = 188.244 | std_rg = 9.518 | std_by = 63.546 | std_rgby = 64.255
european-street-fashion-output/test/test00343.png
european-street-fashion-output/test/test00344.png
european-street-fashion-output/test/test00345.png
european-street-fashion-output/test/test00346.png
european-street-fashion-output/test/test00347.png
european-street-fashion-output/test/test00348.png
european-street-fashion-output/test/test00349.png
european-street-fashion-output/test/test0

...

thai-street-fashion-output/test/test00099.png
thai-street-fashion-output/test/test00100.png
thai-street-fashion-output/test/test00101.png
thai-street-fashion-output/test/test00102.png
thai-street-fashion-output/test/test00103.png
u_rg = 71.159 | u_by = 36.437 | u_rgby = 79.946 | std_rg = 73.322 | std_by = 28.029 | std_rgby = 78.497
thai-street-fashion-output/test/test00104.png
u_rg = 8.562 | u_by = 13.310 | u_rgby = 15.826 | std_rg = 12.002 | std_by = 11.512 | std_rgby = 16.630
thai-street-fashion-output/test/test00105.png
u_rg = 19.116 | u_by = 46.626 | u_rgby = 50.392 | std_rg = 16.183 | std_by = 23.751 | std_rgby = 28.740
thai-street-fashion-output/test/test00106.png
u_rg = 27.429 | u_by = 7.768 | u_rgby = 28.508 | std_rg = 34.562 | std_by = 3.705 | std_rgby = 34.760
thai-street-fashion-output/test/test00107.png
u_rg = 18.503 | u_by = 54.097 | u_rgby = 57.174 | std_rg = 32.649 | std_by = 40.403 | std_rgby = 51.945
thai-street-fashion-output/test/test00108.png
thai-street-fashion-out

u_rg = 11.498 | u_by = 13.729 | u_rgby = 17.908 | std_rg = 20.508 | std_by = 23.190 | std_rgby = 30.958
thai-street-fashion-output/test/test00188.png
u_rg = 31.966 | u_by = 33.135 | u_rgby = 46.041 | std_rg = 33.613 | std_by = 25.873 | std_rgby = 42.418
thai-street-fashion-output/test/test00189.png
thai-street-fashion-output/test/test00190.png
thai-street-fashion-output/test/test00191.png
thai-street-fashion-output/test/test00192.png
thai-street-fashion-output/test/test00193.png
thai-street-fashion-output/test/test00194.png
thai-street-fashion-output/test/test00195.png
thai-street-fashion-output/test/test00196.png
thai-street-fashion-output/test/test00197.png
thai-street-fashion-output/test/test00198.png
thai-street-fashion-output/test/test00199.png
thai-street-fashion-output/test/test00200.png
u_rg = 8.488 | u_by = 10.763 | u_rgby = 13.707 | std_rg = 15.236 | std_by = 15.544 | std_rgby = 21.766
thai-street-fashion-output/test/test00201.png
thai-street-fashion-output/test/test00202.png

u_rg = 5.667 | u_by = 16.352 | u_rgby = 17.306 | std_rg = 8.314 | std_by = 40.725 | std_rgby = 41.565
thai-street-fashion-output/test/test00290.png
u_rg = 6.212 | u_by = 3.059 | u_rgby = 6.925 | std_rg = 15.655 | std_by = 3.958 | std_rgby = 16.148
thai-street-fashion-output/test/test00291.png
thai-street-fashion-output/test/test00292.png
thai-street-fashion-output/test/test00293.png
thai-street-fashion-output/test/test00294.png
u_rg = 10.557 | u_by = 10.889 | u_rgby = 15.166 | std_rg = 11.976 | std_by = 9.963 | std_rgby = 15.578
thai-street-fashion-output/test/test00295.png
thai-street-fashion-output/test/test00296.png
thai-street-fashion-output/test/test00297.png
u_rg = 18.353 | u_by = 41.075 | u_rgby = 44.988 | std_rg = 8.922 | std_by = 18.350 | std_rgby = 20.404
thai-street-fashion-output/test/test00298.png
thai-street-fashion-output/test/test00299.png
thai-street-fashion-output/test/test00300.png
thai-street-fashion-output/test/test00301.png
thai-street-fashion-output/test/test0030

thai-street-fashion-output/test/test00390.png
thai-street-fashion-output/test/test00391.png
thai-street-fashion-output/test/test00392.png
thai-street-fashion-output/test/test00393.png
thai-street-fashion-output/test/test00394.png
thai-street-fashion-output/test/test00395.png
thai-street-fashion-output/test/test00396.png
thai-street-fashion-output/test/test00397.png
thai-street-fashion-output/test/test00398.png
thai-street-fashion-output/test/test00399.png
thai-street-fashion-output/test/test00400.png
thai-street-fashion-output/test/test00401.png
thai-street-fashion-output/test/test00402.png
thai-street-fashion-output/test/test00403.png
thai-street-fashion-output/test/test00404.png
thai-street-fashion-output/test/test00405.png
thai-street-fashion-output/test/test00406.png
thai-street-fashion-output/test/test00407.png
thai-street-fashion-output/test/test00408.png
u_rg = 6.843 | u_by = 11.409 | u_rgby = 13.304 | std_rg = 12.829 | std_by = 15.467 | std_rgby = 20.095
thai-street-fashion-out

thai-street-fashion-output/test/test00492.png
u_rg = 10.675 | u_by = 10.593 | u_rgby = 15.039 | std_rg = 18.136 | std_by = 17.906 | std_rgby = 25.486
thai-street-fashion-output/test/test00493.png
u_rg = 45.412 | u_by = 37.259 | u_rgby = 58.740 | std_rg = 47.331 | std_by = 25.945 | std_rgby = 53.976
thai-street-fashion-output/test/test00494.png
u_rg = 53.532 | u_by = 52.252 | u_rgby = 74.806 | std_rg = 29.837 | std_by = 29.180 | std_rgby = 41.734
thai-street-fashion-output/test/test00495.png
u_rg = 79.022 | u_by = 45.285 | u_rgby = 91.078 | std_rg = 54.574 | std_by = 18.040 | std_rgby = 57.478
thai-street-fashion-output/test/test00496.png
u_rg = 21.931 | u_by = 23.223 | u_rgby = 31.942 | std_rg = 25.629 | std_by = 29.777 | std_rgby = 39.287
thai-street-fashion-output/test/test00497.png
thai-street-fashion-output/test/test00498.png
thai-street-fashion-output/test/test00499.png
thai-street-fashion-output/test/test00500.png
thai-street-fashion-output/test/test00501.png
thai-street-fashion-

In [10]:
test

In [11]:
test

In [12]:
time_in_sec = ed-st
time_h = int(time_in_sec/3600)
time_m = int((time_in_sec-60*time_h)/60)
time_s = time_in_sec-3600*time_h-60*time_m

In [13]:
print('Process Time: {} hr(s). {} min(s) {:.6f} s.'.format(time_h, time_m, time_s))

Process Time: 0 hr(s). 31 min(s) 38.883171 s.


In [14]:
column_names[1:]

['short_sleeved_shirt',
 'long_sleeved_shirt',
 'short_sleeved_outwear',
 'long_sleeved_outwear',
 'vest',
 'sling',
 'shorts',
 'trousers',
 'skirt',
 'short_sleeved_dress',
 'long_sleeved_dress',
 'vest_dress',
 'sling_dress']

In [15]:
class_tab[column_names[1:]] = class_tab[column_names[1:]].astype(float)

In [16]:
class_tab

,filename,short_sleeved_shirt,long_sleeved_shirt,short_sleeved_outwear,long_sleeved_outwear,vest,sling,shorts,trousers,skirt,short_sleeved_dress,long_sleeved_dress,vest_dress,sling_dress
0,european-street-fashion-output/test/test00000.png,0.0,0.000000,0.000000,0.0,0.987999,0.0,0.0,0.000000,0.887868,0.0,0.0,0.0,0.0
0,european-street-fashion-output/test/test00001.png,0.0,0.000000,0.959102,0.0,0.000000,0.0,0.0,0.985530,0.930180,0.0,0.0,0.0,0.0
0,european-street-fashion-output/test/test00005.png,0.0,0.000000,0.000000,0.0,0.998924,0.0,0.0,0.894071,0.000000,0.0,0.0,0.0,0.0
0,european-street-fashion-output/test/test00006.png,0.0,0.000000,0.770587,0.0,0.996589,0.0,0.0,0.000000,0.939734,0.0,0.0,0.0,0.0
0,european-street-fashion-output/test/test00008.png,0.0,0.000000,0.000000,0.0,0.970125,0.0,0.0,0.732579,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,thai-street-fashion-output/test/test00515.png,0.0,0.000000,0.000000,0.0,0.952845,0.0,0.0,0.000000,0.993711,0.0,0.0,0.0,0.0
0,thai-street-fashion-output/test/test00516.png,0.0,0.000000,0.879736,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
0,thai-street-fashion-output/test/test00526.png,0.0,0.000000,0.757502,0.0,0.786874,0.0,0.0,0.877317,0.000000,0.0,0.0,0.0,0.0
0,thai-street-fashion-output/test/test00527.png,0.0,0.845655,0.000000,0.0,0.000000,0.0,0.0,0.999213,0.000000,0.0,0.0,0.0,0.0


In [17]:
top7_tab

,filename,r_range_1,g_range_1,b_range_1,proportion_1,r_range_2,g_range_2,b_range_2,proportion_2,r_range_3,g_range_3,b_range_3,proportion_3,r_range_4,g_range_4,b_range_4,proportion_4,r_range_5,g_range_5,b_range_5,proportion_5,r_range_6,g_range_6,b_range_6,proportion_6,r_range_7,g_range_7,b_range_7,proportion_7,label
0,european-street-fashion-output/test/test00000.png,16-31,16-31,16-31,0.303225,0-15,0-15,0-15,0.235109,0-15,0-15,16-31,0.050533,0-15,16-31,16-31,0.040913,16-31,16-31,32-47,0.026910,96-111,128-143,176-191,0.020603,96-111,144-159,192-207,0.018866,0
0,european-street-fashion-output/test/test00001.png,16-31,16-31,16-31,0.176878,240-255,240-255,240-255,0.091829,224-239,224-239,224-239,0.076498,0-15,0-15,0-15,0.073471,224-239,224-239,240-255,0.039134,224-239,240-255,240-255,0.035281,208-223,208-223,208-223,0.028382,0
0,european-street-fashion-output/test/test00005.png,224-239,224-239,224-239,0.092203,208-223,208-223,208-223,0.082258,224-239,224-239,240-255,0.065342,192-207,192-207,192-207,0.046121,80-95,80-95,80-95,0.039306,176-191,176-191,176-191,0.039293,64-79,64-79,64-79,0.038894,0
0,european-street-fashion-output/test/test00006.png,48-63,32-47,32-47,0.063760,112-127,32-47,0-15,0.059953,128-143,48-63,0-15,0.055638,144-159,48-63,0-15,0.042434,96-111,32-47,0-15,0.036150,240-255,240-255,240-255,0.033036,64-79,48-63,48-63,0.032069,0
0,european-street-fashion-output/test/test00008.png,0-15,0-15,0-15,0.127613,192-207,176-191,160-175,0.112392,192-207,176-191,144-159,0.067298,208-223,192-207,176-191,0.067047,192-207,160-175,144-159,0.059518,0-15,16-31,16-31,0.050183,0-15,0-15,16-31,0.035422,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,thai-street-fashion-output/test/test00515.png,0-15,0-15,16-31,0.382456,0-15,16-31,16-31,0.152202,112-127,96-111,96-111,0.019563,0-15,16-31,32-47,0.019206,96-111,80-95,80-95,0.018939,128-143,112-127,112-127,0.018107,80-95,64-79,64-79,0.015809,0
0,thai-street-fashion-output/test/test00516.png,32-47,32-47,32-47,0.196324,192-207,192-207,192-207,0.066635,176-191,176-191,176-191,0.037899,208-223,208-223,208-223,0.029360,16-31,16-31,16-31,0.028479,176-191,192-207,192-207,0.026092,48-63,48-63,48-63,0.024403,0
0,thai-street-fashion-output/test/test00526.png,64-79,112-127,128-143,0.030152,48-63,112-127,144-159,0.028196,64-79,112-127,144-159,0.026224,48-63,96-111,112-127,0.024860,48-63,96-111,128-143,0.023839,64-79,128-143,160-175,0.023400,32-47,96-111,128-143,0.022496,0
0,thai-street-fashion-output/test/test00527.png,32-47,32-47,0-15,0.134387,32-47,32-47,16-31,0.129191,16-31,32-47,16-31,0.048654,144-159,112-127,64-79,0.043070,48-63,48-63,32-47,0.039828,128-143,112-127,48-63,0.032310,48-63,48-63,48-63,0.024963,0


In [18]:
colorful_tab

,filename,colorfulness
0,european-street-fashion-output/test/test00000.png,35.53998634401716
0,european-street-fashion-output/test/test00001.png,13.672892124321166
0,european-street-fashion-output/test/test00005.png,9.019222735728928
0,european-street-fashion-output/test/test00006.png,64.19510766989845
0,european-street-fashion-output/test/test00008.png,25.58021030931733
...,...,...
0,thai-street-fashion-output/test/test00515.png,21.898129614672417
0,thai-street-fashion-output/test/test00516.png,19.477787076231508
0,thai-street-fashion-output/test/test00526.png,44.89309654149794
0,thai-street-fashion-output/test/test00527.png,38.425786438389075


In [19]:
top7_tab = pd.merge(top7_tab, class_tab, on='filename', how='inner')
top7_tab = pd.merge(top7_tab, colorful_tab, on='filename', how='inner')

In [20]:
top7_tab

,filename,r_range_1,g_range_1,b_range_1,proportion_1,r_range_2,g_range_2,b_range_2,proportion_2,r_range_3,g_range_3,b_range_3,proportion_3,r_range_4,g_range_4,b_range_4,proportion_4,r_range_5,g_range_5,b_range_5,proportion_5,r_range_6,g_range_6,b_range_6,proportion_6,r_range_7,g_range_7,b_range_7,proportion_7,label,short_sleeved_shirt,long_sleeved_shirt,short_sleeved_outwear,long_sleeved_outwear,vest,sling,shorts,trousers,skirt,short_sleeved_dress,long_sleeved_dress,vest_dress,sling_dress,colorfulness
0,european-street-fashion-output/test/test00000.png,16-31,16-31,16-31,0.303225,0-15,0-15,0-15,0.235109,0-15,0-15,16-31,0.050533,0-15,16-31,16-31,0.040913,16-31,16-31,32-47,0.026910,96-111,128-143,176-191,0.020603,96-111,144-159,192-207,0.018866,0,0.0,0.000000,0.000000,0.0,0.987999,0.0,0.0,0.000000,0.887868,0.0,0.0,0.0,0.0,35.53998634401716
1,european-street-fashion-output/test/test00001.png,16-31,16-31,16-31,0.176878,240-255,240-255,240-255,0.091829,224-239,224-239,224-239,0.076498,0-15,0-15,0-15,0.073471,224-239,224-239,240-255,0.039134,224-239,240-255,240-255,0.035281,208-223,208-223,208-223,0.028382,0,0.0,0.000000,0.959102,0.0,0.000000,0.0,0.0,0.985530,0.930180,0.0,0.0,0.0,0.0,13.672892124321166
2,european-street-fashion-output/test/test00005.png,224-239,224-239,224-239,0.092203,208-223,208-223,208-223,0.082258,224-239,224-239,240-255,0.065342,192-207,192-207,192-207,0.046121,80-95,80-95,80-95,0.039306,176-191,176-191,176-191,0.039293,64-79,64-79,64-79,0.038894,0,0.0,0.000000,0.000000,0.0,0.998924,0.0,0.0,0.894071,0.000000,0.0,0.0,0.0,0.0,9.019222735728928
3,european-street-fashion-output/test/test00006.png,48-63,32-47,32-47,0.063760,112-127,32-47,0-15,0.059953,128-143,48-63,0-15,0.055638,144-159,48-63,0-15,0.042434,96-111,32-47,0-15,0.036150,240-255,240-255,240-255,0.033036,64-79,48-63,48-63,0.032069,0,0.0,0.000000,0.770587,0.0,0.996589,0.0,0.0,0.000000,0.939734,0.0,0.0,0.0,0.0,64.19510766989845
4,european-street-fashion-output/test/test00008.png,0-15,0-15,0-15,0.127613,192-207,176-191,160-175,0.112392,192-207,176-191,144-159,0.067298,208-223,192-207,176-191,0.067047,192-207,160-175,144-159,0.059518,0-15,16-31,16-31,0.050183,0-15,0-15,16-31,0.035422,0,0.0,0.000000,0.000000,0.0,0.970125,0.0,0.0,0.732579,0.000000,0.0,0.0,0.0,0.0,25.58021030931733
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
602,thai-street-fashion-output/test/test00515.png,0-15,0-15,16-31,0.382456,0-15,16-31,16-31,0.152202,112-127,96-111,96-111,0.019563,0-15,16-31,32-47,0.019206,96-111,80-95,80-95,0.018939,128-143,112-127,112-127,0.018107,80-95,64-79,64-79,0.015809,0,0.0,0.000000,0.000000,0.0,0.952845,0.0,0.0,0.000000,0.993711,0.0,0.0,0.0,0.0,21.898129614672417
603,thai-street-fashion-output/test/test00516.png,32-47,32-47,32-47,0.196324,192-207,192-207,192-207,0.066635,176-191,176-191,176-191,0.037899,208-223,208-223,208-223,0.029360,16-31,16-31,16-31,0.028479,176-191,192-207,192-207,0.026092,48-63,48-63,48-63,0.024403,0,0.0,0.000000,0.879736,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,19.477787076231508
604,thai-street-fashion-output/test/test00526.png,64-79,112-127,128-143,0.030152,48-63,112-127,144-159,0.028196,64-79,112-127,144-159,0.026224,48-63,96-111,112-127,0.024860,48-63,96-111,128-143,0.023839,64-79,128-143,160-175,0.023400,32-47,96-111,128-143,0.022496,0,0.0,0.000000,0.757502,0.0,0.786874,0.0,0.0,0.877317,0.000000,0.0,0.0,0.0,0.0,44.89309654149794
605,thai-street-fashion-output/test/test00527.png,32-47,32-47,0-15,0.134387,32-47,32-47,16-31,0.129191,16-31,32-47,16-31,0.048654,144-159,112-127,64-79,0.043070,48-63,48-63,32-47,0.039828,128-143,112-127,48-63,0.032310,48-63,48-63,48-63,0.024963,0,0.0,0.845655,0.000000,0.0,0.000000,0.0,0.0,0.999213,0.000000,0.0,0.0,0.0,0.0,38.425786438389075


In [21]:
th = top7_tab[top7_tab['filename']=='thai-street-fashion-output/test/test00066.png']
jp = top7_tab[top7_tab['filename']=='harajuku-kawaii-street-fashion-output/test/test00011.png']

In [22]:
pd.concat([jp, th]).to_csv('example.csv', index=False)

In [23]:
top7_tab = top7_tab[[c for c in top7_tab if c not in ['label']]+['label']]

In [24]:
top7_tab['label'].value_counts()

0    455
1    152
Name: label, dtype: int64

In [25]:
top7_tab.to_csv('rgb_img_dataset_test_colorfulness.csv', index=False)

In [26]:
top7_tab['colorfulness'] = top7_tab['colorfulness'].astype(float)

In [27]:
top7_tab.sort_values(['colorfulness'], ascending=False)

,filename,r_range_1,g_range_1,b_range_1,proportion_1,r_range_2,g_range_2,b_range_2,proportion_2,r_range_3,g_range_3,b_range_3,proportion_3,r_range_4,g_range_4,b_range_4,proportion_4,r_range_5,g_range_5,b_range_5,proportion_5,r_range_6,g_range_6,b_range_6,proportion_6,r_range_7,g_range_7,b_range_7,proportion_7,short_sleeved_shirt,long_sleeved_shirt,short_sleeved_outwear,long_sleeved_outwear,vest,sling,shorts,trousers,skirt,short_sleeved_dress,long_sleeved_dress,vest_dress,sling_dress,colorfulness,label
366,harajuku-kawaii-street-fashion-output/test/test00273.png,240-255,96-111,224-239,0.043350,0-15,208-223,240-255,0.034274,240-255,240-255,240-255,0.030577,0-15,192-207,240-255,0.027005,0-15,176-191,240-255,0.023182,240-255,112-127,240-255,0.021983,0-15,224-239,240-255,0.020443,0.0,0.000000,0.838560,0.0,0.965593,0.0,0.0,0.956809,0.000000,0.000000,0.0,0.0,0.0,140.270912,1
14,european-street-fashion-output/test/test00023.png,240-255,0-15,48-63,0.169258,224-239,0-15,32-47,0.059531,224-239,0-15,48-63,0.050143,208-223,0-15,32-47,0.045567,192-207,0-15,32-47,0.030147,240-255,16-31,64-79,0.026930,240-255,0-15,32-47,0.018241,0.0,0.000000,0.000000,0.0,0.998953,0.0,0.0,0.982705,0.866159,0.000000,0.0,0.0,0.0,133.851451,0
278,harajuku-kawaii-street-fashion-output/test/test00042.png,0-15,176-191,224-239,0.044277,240-255,144-159,208-223,0.038863,240-255,128-143,208-223,0.036464,240-255,0-15,128-143,0.034249,240-255,112-127,192-207,0.030895,240-255,160-175,224-239,0.030540,0-15,192-207,224-239,0.030090,0.0,0.000000,0.905243,0.0,0.821585,0.0,0.0,0.000000,0.000000,0.894529,0.0,0.0,0.0,133.319131,1
23,european-street-fashion-output/test/test00038.png,240-255,48-63,16-31,0.133088,240-255,32-47,0-15,0.089954,240-255,64-79,32-47,0.082212,240-255,48-63,0-15,0.055484,240-255,16-31,0-15,0.035760,240-255,48-63,32-47,0.022304,240-255,64-79,16-31,0.018433,0.0,0.000000,0.000000,0.0,0.986597,0.0,0.0,0.849035,0.000000,0.000000,0.0,0.0,0.0,130.791757,0
339,harajuku-kawaii-street-fashion-output/test/test00186.png,0-15,0-15,0-15,0.137826,240-255,224-239,0-15,0.020316,0-15,176-191,208-223,0.017183,240-255,208-223,0-15,0.016419,0-15,192-207,208-223,0.015681,240-255,224-239,48-63,0.014170,240-255,224-239,64-79,0.013489,0.0,0.000000,0.887390,0.0,0.853626,0.0,0.0,0.989723,0.000000,0.729876,0.0,0.0,0.0,130.783282,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224,european-street-fashion-output/test/test00533.png,0-15,0-15,0-15,0.726672,16-31,16-31,16-31,0.109036,16-31,16-31,32-47,0.029298,0-15,0-15,16-31,0.024615,32-47,32-47,48-63,0.024226,32-47,32-47,32-47,0.018888,48-63,48-63,48-63,0.009711,0.0,0.000000,0.738435,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,5.010066,0
168,european-street-fashion-output/test/test00364.png,0-15,0-15,0-15,0.457367,16-31,16-31,16-31,0.146917,144-159,144-159,144-159,0.139739,144-159,144-159,160-175,0.047563,128-143,128-143,144-159,0.034054,128-143,128-143,128-143,0.026934,160-175,160-175,160-175,0.021430,0.0,0.000000,0.000000,0.0,0.998996,0.0,0.0,0.000000,0.974582,0.000000,0.0,0.0,0.0,4.469051,0
223,european-street-fashion-output/test/test00532.png,16-31,16-31,16-31,0.226686,0-15,0-15,0-15,0.205210,32-47,32-47,32-47,0.187731,16-31,16-31,32-47,0.102240,32-47,32-47,48-63,0.077138,0-15,0-15,16-31,0.059190,48-63,48-63,48-63,0.043736,0.0,0.000000,0.726165,0.0,0.711747,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,4.130868,0
240,european-street-fashion-output/test/test00590.png,0-15,0-15,0-15,0.342432,16-31,16-31,16-31,0.137304,32-47,32-47,32-47,0.127468,48-63,48-63,48-63,0.078069,16-31,16-31,0-15,0.076855,48-63,48-63,32-47,0.037770,32-47,32-47,16-31,0.035720,0.0,0.000000,0.740806,0.0,0.908773,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,3.745218,0


In [28]:
top7_tab['colorfulness']

0      35.539986
1      13.672892
2       9.019223
3      64.195108
4      25.580210
         ...    
602    21.898130
603    19.477787
604    44.893097
605    38.425786
606    83.378443
Name: colorfulness, Length: 607, dtype: float64

In [29]:
f = open('eu-seg/test/class_00000.txt', 'r')
classProb = f.readlines()[0].split(' ')[:-1]

In [30]:
classProb

['0',
 '0',
 '0',
 '0',
 '0.98799884',
 '0',
 '0',
 '0',
 '0.8878681',
 '0',
 '0',
 '0',
 '0']

In [31]:
class_names = ['short_sleeved_shirt', 'long_sleeved_shirt', 'short_sleeved_outwear', 'long_sleeved_outwear', 'vest', 'sling',
               'shorts', 'trousers', 'skirt', 'short_sleeved_dress', 'long_sleeved_dress', 'vest_dress', 'sling_dress']
column_names = ['filename']+class_names

In [32]:
column_names

['filename',
 'short_sleeved_shirt',
 'long_sleeved_shirt',
 'short_sleeved_outwear',
 'long_sleeved_outwear',
 'vest',
 'sling',
 'shorts',
 'trousers',
 'skirt',
 'short_sleeved_dress',
 'long_sleeved_dress',
 'vest_dress',
 'sling_dress']